# Python Programming Practice
## Chapter 08. 정규 표현식 Regular expression

### 08-1. 정규표현식 살펴보기

In [1]:
# 주민등록번호를 포함하는 텍스트에 포함된 모든 주민등록번호의 뒷자리를 *문자로 변경

def number_to_star(data):
    result = []
    for line in data.split("\n"):
        word_result = []
        for word in line.split(" "):
            # 뒷자리를 *로 변경
            if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
                word = word[:6] + "-" + "*" * 7
            # " "로 구분된 모든 단어 저장
            word_result.append(word)
        # 단어들을 묶어서 result에 저장
        result.append(" ".join(word_result))

    print("\n".join(result))

data = """
park 800905-1049118
kim 980901-2578123
"""

number_to_star(data)


park 800905-*******
kim 980901-*******



In [2]:
# regular expression 사용
import re

pat = re.compile(r"(\d{6})[-]\d{7}")
print(pat.sub(r"\g<1>-*******", data))


park 800905-*******
kim 980901-*******



### 08-2. 정규표현식 시작하기

#### 정규 표현식의 기초, 메타 문자
> . ^ $ + ? {} [] \ | ()

In [3]:
import re

def print_match(reg_pattern, text):
    p = re.compile(reg_pattern)

    print(p.match(text))

In [4]:
# 1. []: character class

# [abc]: a, b, c중 하나라도 매치
print_match(r"[abc]", "a")
print_match(r"[abc]", "before")
print_match(r"[abc]", "dude")

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='b'>
None


In [5]:
# [a-zA-Z]: 모든 문자, [0-9]: 모든 숫자
print_match(r"\d", "34") # \d == [0-9]
print_match(r"\D", "str") # \D == [^0-9]
print_match(r"\s", "a\nb") # \s == [ \t\n\r\f\v], \S == [ ^\t\n...\v]
print_match(r"\s", "\n") 
print_match(r"\w", "my 8 goals") # \w == [a-zA-Z0-9], white space 전까지 검색
print_match(r"\W", "my 8 goals") # \W == [^a-zA-Z0-9], white space 전까지 검색

<re.Match object; span=(0, 1), match='3'>
<re.Match object; span=(0, 1), match='s'>
None
<re.Match object; span=(0, 1), match='\n'>
<re.Match object; span=(0, 1), match='m'>
None


In [6]:
# 2. .(dot): \n을 제외한 모든 문자 하나
print_match(r"a.b", "a1b") # O
print_match(r"a.b", "aab") # O
print_match(r"a.b", "abc") # X
print_match(r"a.b", "avvb") # X


<re.Match object; span=(0, 3), match='a1b'>
<re.Match object; span=(0, 3), match='aab'>
None
None


In [7]:
# 3. *: {0, inf} 반복
print_match(r"ca*t", "ct") # O
print_match(r"ca*t", "cat") # O
print_match(r"ca*t", "caat") # O
print_match(r"ca*t", "cdt") # X

<re.Match object; span=(0, 2), match='ct'>
<re.Match object; span=(0, 3), match='cat'>
<re.Match object; span=(0, 4), match='caat'>
None


In [8]:
# 4. + : {1, inf} 반복
print_match(r"ca+t", "ct") # X
print_match(r"ca+t", "cat") # O
print_match(r"ca+t", "caat") # O
print_match(r"ca+t", "cdt") # X

None
<re.Match object; span=(0, 3), match='cat'>
<re.Match object; span=(0, 4), match='caat'>
None


In [9]:
# 5. {m,n}: m <= i <= n, i번 반복 / {m}: m번 반복
print_match(r"ca{2,5}t", "cat") # X
print_match(r"ca{2,5}t", "caat") # O
print_match(r"ca{2,5}t", "caaaaat") # O
print_match(r"ca{2,5}t", "cbt") # X
print_match(r"ca{2}t", "cat") # X
print_match(r"ca{2}t", "caat") # O


None
<re.Match object; span=(0, 4), match='caat'>
<re.Match object; span=(0, 7), match='caaaaat'>
None
None
<re.Match object; span=(0, 4), match='caat'>


In [10]:
# 6. ? : {0, 1}, 있거나 없어도 됨
# r"ab?c" == "abc" or "ac"
print_match(r"ab?c", "abc") # O
print_match(r"ab?c", "ac") # O
print_match(r"ab?c", "arc") # X

<re.Match object; span=(0, 3), match='abc'>
<re.Match object; span=(0, 2), match='ac'>
None


#### 정규식을 이용한 문자열 검색

`p = re.compile(r"regular expression pattern")`

- 컴파일 된 패턴 객체를 사용하여 문자열 검색 수행

1. `match()`: 문자열 처음부터 정규식패턴과 매치되는지 조사, return match obj or None
2. `search()`: 문자열 전체를 검색하여 정규식패턴과 매치되는지 조사, return match obj or None
3. `findall()`: 정규식패턴과 매치되는 모든 문자열(substring)을 list로 return, item은 string
4. `finditer()`: 정규식패턴과 매치되는 모든 문자열(substring)을 iterator 객체로 return, item은 match 객체

In [11]:
import re

# 영어 소문자가 하나 이상 반복
p = re.compile(r"[a-z]+") 

In [12]:
# 1. match(): 문자열 처음부터 조사 
m = p.match("python")
print(m)

<re.Match object; span=(0, 6), match='python'>


In [13]:
m = p.match("3 python") # while space 이전까지 조사
print(m)

None


In [14]:
# 2. search(): 문자열 전체 조사
m = p.search("python")
print(m)

m = p.search("3 python") # white space 이후 문자열까지 모두 조사
print(m)

<re.Match object; span=(0, 6), match='python'>
<re.Match object; span=(2, 8), match='python'>


In [15]:
# 3. findall(): match되는 모든 문자열(str) list로 return
result = p.findall("life is too short, I Like Python")
print(result)

['life', 'is', 'too', 'short', 'ike', 'ython']


In [16]:
# 4. finditer(): match되는 모든 문자열(match 객체) iterator로 return
result = p.finditer("life is too short")
print(result)

for item in result:
    print(item)

<re.Match object; span=(0, 4), match='life'>
<re.Match object; span=(5, 7), match='is'>
<re.Match object; span=(8, 11), match='too'>
<re.Match object; span=(12, 17), match='short'>


#### `match` 객체의 메서드
1. `group()`: 매치된 문자열 return
2. `start()`: 매치된 문자열 시작 위치 return
3. `end()`: 매치된 문자열 끝 위치 return
4. `span()`: 매치된 문자열 (start, end) 튜플 리턴

In [17]:
p = re.compile(r"[a-z]+")
m = p.match("python")

print(m.group())
print(m.start())
print(m.end())
print(m.span())

python
0
6
(0, 6)


In [18]:
m = p.search("3 python")
print(m.group())
print(m.start())
print(m.end())
print(m.span())

python
2
8
(2, 8)


In [19]:
# 모듈 단위로 수행하기
p = re.compile(r"[a-z]+")
m = p.match("python")

print(m)

<re.Match object; span=(0, 6), match='python'>


In [20]:
m = re.match(r"[a-z]+", "python")
print(m)

<re.Match object; span=(0, 6), match='python'>


#### 컴파일 옵션
1. `DOTALL` == `S`: .(dot)이 \n을 포함해 모든 문자와 매칭
2. `IGNORECASE` == `I`: 대소문자 관계없이 매칭
3. `MULTILINE` == `M`: 여러 줄과 매칭, `^`, `$` 메타문자 각 줄마다 적용
4. `VERBOSE` == `X`: verbose 모드 사용, 정규식 내부 white space 무시하여 주석등 추가 가능

In [21]:
# 1. DOTALL, S : .(dot)이 \n 포함 
# > 여러줄 문자열에서 줄바꿈과 상관없이 검색할 때 사용
p = re.compile(r"a.b")
m = p.match("a\nb")

print(m)

None


In [22]:
p = re.compile(r"a.b", re.DOTALL)
m = p.match("a\nb")

print(m)

<re.Match object; span=(0, 3), match='a\nb'>


In [23]:
# 2. IGNORECASE, I: 대소문자 구분 없이 매칭
p = re.compile(r"[a-z]+", re.I)
print(p.match("python"))
print(p.match("Python"))
print(p.match("PYTHON"))

<re.Match object; span=(0, 6), match='python'>
<re.Match object; span=(0, 6), match='Python'>
<re.Match object; span=(0, 6), match='PYTHON'>


In [24]:
# 3. MULTILINE, M: 여러줄과 매칭
# ^, $ 메타 문자를 각 줄마다 적용
import re

# python으로 시작, 화이트 스페이스 이후 어떤 문자든 와도 됨
p = re.compile(r"^python\s\w+")

data = '''python one
life is too short
python two
you need python
python three
'''

print(p.findall(data))

['python one']


In [25]:
# 각 라인별로 매칭
p = re.compile(r"^python\s\w+", re.M)

print(p.findall(data))

['python one', 'python two', 'python three']


In [26]:
# 4. VERBOSE, X: 정규식 이해할 수 있도록 변환 가능, 정규식 안의 white space 무시

# 이해하기 어려움
charref1 = re.compile(r"&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);")

In [27]:
# VERBOSE 옵션 적용, white space 무시, 주석 달기 및 라인 구분 가능
charref2 = re.compile(
r"""
&[#]    # Start of a numeric entity reference
(
0[0-7]+     # Octal form
| [0-9]+    # Decimal form
| x[0-9a-fA-F]+ # Hecadecimal form
)
;                         
""", re.VERBOSE)

### 08-3. 강력한 정규 표현식의 세계로

#### 문자열 소비가 없는 메타 문자(zerowidth assertion)

In [28]:
# | : or
import re

p = re.compile("Crow|Servo")
m = p.match("CrowHello")

print(m)

<re.Match object; span=(0, 4), match='Crow'>


In [29]:
# ^: 문자열의 맨 처음과 일치
print(re.search("^Life", "Life is too short"))
print(re.search("^Life", "My Life"))

<re.Match object; span=(0, 4), match='Life'>
None


In [30]:
# $: 문자열 맨 끝과 일치
print(re.search("short$", "Life is too short"))
print(re.search("short$", "Life is too short, you need python"))

<re.Match object; span=(12, 17), match='short'>
None


In [31]:
# \A: 문자열의 맨 처음과 일치, 단 re.MULTILINE 옵션 사용시 ^는 모든 line의 맨처음을 따지지만 \A는 전체문자열의 처음만 따진다
# \Z: 문자열의 맨 마지막과 일치, 단 re.MULTILINE 옵션 사용시 $는 모든 line의 맨 마지막을 따지지만 \Z는 전체 문자열의 마지막만 따진다.

In [32]:
# \b: word boundary, 단어 구분자, 화이트 스페이스로 구분됨
p = re.compile(r"\bclass\b")
print(p.search("no class at all"))

<re.Match object; span=(3, 8), match='class'>


In [33]:
print(p.search("the declassified algorithm"))

None


In [34]:
print(p.search("one subclass is"))

None


In [35]:
# \B: 화이트 스페이스로 구분되지 않은 경우에만 매치
p = re.compile(r"\Bclass\B")
print(p.search("no class at all"))
print(p.search("the declassified algorithm"))
print(p.search("one subclass is"))

None
<re.Match object; span=(6, 11), match='class'>
None


#### 그루핑
- 여러 문자에 대해 조사 () 그루핑 사용

In [36]:
# ABC 문자열이 계속해서 반복되는지 조사하는 정규식
p = re.compile(r"(ABC)+")
m = p.search("ABCABCABCAB OK?")

print(m)
print(m.group())

<re.Match object; span=(0, 9), match='ABCABCABC'>
ABCABCABC


In [37]:
# 이름 + " " + 전화번호 형태를 찾는 정규식
p = re.compile(r"\w+\s+\d+[-]\d+[-]\d+")
m = p.search("park 019-1234-5678")
print(m, m.group())

<re.Match object; span=(0, 18), match='park 019-1234-5678'> park 019-1234-5678


In [38]:
# 특정 부분만 뽑아내기 > grouping() 후 인덱스 지정
p = re.compile(r"(\w+)\s+(\d+[-]\d+[-]\d+)")
m = p.search("park 019-1234-5678")
print(m.group(1))
print(m.group(2))

park
019-1234-5678


In [39]:
# grouping of grouping, 전화번호 맨처음만 추출하기
p = re.compile(r"(\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("Kim 013-1234-5679")

print(m.group(3))

013


In [40]:
# 그루핑의 문자열 재참조(back-references)하기
p = re.compile(r"(\b\w+)\s+\1")
m = p.search("Paris in the the spring")

print(m)

<re.Match object; span=(9, 16), match='the the'>


In [41]:
# 그루핑된 문자열에 이름 붙이기: (?P<그룹명>메타문자)
p = re.compile(r"(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("Jang 011-1234-5678")

print(m.group("name"))

Jang


In [42]:
# 그룹 이름을 사용한 재참조: (?P=그룹명)
p = re.compile(r"(?P<word>\b\w+)\s+(?P=word)")
p.search("Paris in the the rain.").group()

'the the'

#### 전방 탐색(lookahead assertions)

In [43]:
p = re.compile(r".+:")
m = p.search("http://google.com")
print(m.group())

http:


- 긍정형 전방탐색`((?=...))`: ...에 해당하는 정규식과 매치되어야 하며 조건이 통과되어도 문자열이 소비되지 않는다
- 부정형 전방탐색`((?!...))`: ...에 해당하는 정규식과 매치되지 않아야 하며 조건이 통과되어도 문자열이 소비되지 않는다

In [44]:
# 긍정형 전방탐색, http: > http
# :에 해당하는 문자열이 정규식 엔진에 의해 소비되지 않아, 검색에는 포함되지만, 검색 결과에서는 제외된다.
p = re.compile(".+(?=:)")
m = p.search("http://google.com")

print(m.group())

http


In [45]:
# 부정형 전방탐색
p = re.compile(r".*[.].*$") # 파일이름.확장자 문자열 검색
m = p.search("test.mp temp.bat myfile.bar")
print(m.group())

test.mp temp.bat myfile.bar


In [46]:
# 확장자가 .bat인 파일 제외
p = re.compile(r".*[.][^b].*") # b로 시작하는 확장자 제외, 단 .bar파일도 걸러짐 (X)
m = p.search("myfile.bar")
print(m)

p = re.compile(r".*[.]([^b]..|.[^a].|..[^t])") # .bat만 거르고 .bar는 제외, 다만 확장자가 2개의 문자일경우 포함하지 못함 (X)
m = p.search("myfile.bar")
print(m)
m = p.search("temp.bat")
print(m)
m = p.search("myfile.cd")
print(m)

None
<re.Match object; span=(0, 10), match='myfile.bar'>
None
None


In [47]:
# 확장자의 문자개수가 2개여도 통과되는 정규식 > 매우 복잡
p = re.compile(r".*[.]([^b].?.?|.[^a].?.?|..?[^t]?)$")
m = p.search("myfile.cd")
print(m)
m = p.search("temp.bat")
print(m)

<re.Match object; span=(0, 9), match='myfile.cd'>
None


In [48]:
# 부정형 전방탐색 사용
# .bat 또는 .exe 제외
p = re.compile(r".*[.](?!bat$|exe$).*$")

# 개선 버전, 부정형 전방탐색 뒤에 greedy 표현 사용 자제
p = re.compile(r"\w+[.](?!bat|exe)\w+")
m = p.findall("my.bat your.bar his.exe her.dat ")
print(m)

['your.bar', 'her.dat']


#### 문자열 바꾸기

In [49]:
# sub() 메서드 사용, 문자열 바꾸기
p = re.compile(r"(blue|white|red)")
print(p.sub("colour", "blue socks and red shoes"))

colour socks and colour shoes


In [50]:
# p.sub(replacement, dst string)
# 바꾸기 횟수 제어: count=1 파라미터 추가
p.sub("colour", "blue socks and red shoes", count=1)

'colour socks and red shoes'

In [51]:
# sub() 메서드에서 참조구문 사용하기: \g<group name> or \g<index>
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")
print(p.sub(r"\g<phone> \g<name>", "Kim 011-1234-5678"))

print(p.sub(r"\g<2> \g<1>", "Jang 012-2345-6789"))

011-1234-5678 Kim
012-2345-6789 Jang


In [52]:
# sub() 메서드의 parameter로 함수 넣기
def hex_replace(match):
    value = int(match.group())
    return hex(value)

p = re.compile(r"\d+")
p.sub(hex_replace, "Call 65490 for printing, 49152 for user code.")

'Call 0xffd2 for printing, 0xc000 for user code.'

#### greedy와 non-greedy

In [53]:
s = "<html><head><title>My Title</title></head></html>"
len(s)

49

In [54]:
m = re.match("<.*>", s)
print(m.span())
print(m.group())

(0, 49)
<html><head><title>My Title</title></head></html>


In [55]:
# * 메타문자 > greedy 검색 > 문자열 모두 소비
# non-greedy 문자 ? 사용
print(re.match("<.*?>", s).group())

<html>
